# Explore the Global Forecast System (GFS) with hvplot 
[Hvplot](https://hvplot.holoviz.org) is part of the [HoloViz tool suite](holoviz.org) and has awesome capabilities for visualizing Xarray and Pandas data. Here we access the GFS weather forecast data from the Unidata THREDDS Server and visualize it in just a few lines.  We also deal with a couple of problems that people commonly encounter! 

In [ ]:
import xarray as xr
import hvplot.xarray

In [ ]:
url = 'https://thredds-jumbo.unidata.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p25deg/Best'

In [ ]:
ds = xr.open_dataset(url)

hvplot likes longitudes in the range [-180,180], not [0, 360]

In [ ]:
ds = ds.assign_coords(lon=(((ds.lon + 180) % 360) - 180)).sortby('lon')

In [ ]:
ds['u-component_of_wind_height_above_ground']

#### Visualize the GFS in the CONUS region in geographic coordinates

We want to select the CONUS, but we see from the previous cell that latitude values are decreasing, so we slice 50 to 20 instead of 20 to 50. 

In [ ]:
ds_conus = ds.sel(lon=slice(-130,-60), lat=slice(50,20))

We need to tell hvplot which coordinates to use as axes, and `geo=True` says that these are geographic (lon,lat) coordinates

In [ ]:
var = 'u-component_of_wind_height_above_ground'

In [ ]:
forecast = ds_conus[var].hvplot(x='lon', y='lat', geo=True, coastline=True, rasterize=True)

In [ ]:
forecast

Add State outlines

In [ ]:
import geoviews.feature as gvf
from geoviews import opts

In [ ]:
forecast * gvf.states(fill_alpha=0)

#### Visualize the GFS globally using an Orthographic projection and a Panel layout

In [ ]:
import panel as pn
import cartopy.crs as ccrs

In [ ]:
crs = ccrs.Orthographic(central_longitude=-70, central_latitude=30)

In [ ]:
import datetime

In [ ]:
now = datetime.datetime.utcnow()

Select from now to furthest forecast, and specify a selection widget for time values

In [ ]:
viz = ds[var].sel(time=slice(now,None))[:,:].hvplot(x='lon', y='lat', 
                    cmap='rainbow', rasterize=True, projection=crs, coastline=True,
                    title='GFS Explorer', hover=False)
viz = pn.panel(viz, widgets={'time': pn.widgets.Select} )
pn.Column(viz).servable('GFS Explorer')

Specify an animation widget ("scrubber") for time values

In [ ]:
viz = ds[var].sel(time=slice(now,None))[:,:].hvplot(x='lon', y='lat', 
                    cmap='rainbow', rasterize=True, projection=crs, coastline=True, hover=False,
                    widget_type='scrubber', title='GFS Explorer', widget_location='bottom')
pn.Column(viz).servable('GFS Explorer')

Display the time value closest to current time (time=now)

In [ ]:
viz = ds[var].sel(time=now, method='nearest')[:,:].hvplot(x='lon', y='lat',                             
                    cmap='rainbow', rasterize=True, projection=crs, coastline=True,
                    title='GFS Explorer', hover=False)
viz = pn.panel(viz, widgets={'time': pn.widgets.Select} )
pn.Column(viz).servable('GFS Explorer')